In [2]:
import numpy as np
import nibabel as nib
import os
import glob
import gzip
import shutil
from sklearn.preprocessing import MinMaxScaler

# Function to get edema voxels
def get_edema_voxels(image_file, segmentation_mask_file):
    try:
        image = nib.load(image_file).get_fdata()
        segmentation_mask = nib.load(segmentation_mask_file).get_fdata()
        edema_voxels = image[segmentation_mask == 2]
        return edema_voxels.reshape(-1, 1)
    except Exception as e:
        print(f"Error processing file {image_file}: {e}")
        return np.array([])

def extract_common_prefix(filename):
    parts = filename.split('/')
    name = '_'.join(parts[9:11]).split('_')
    return '_'.join(name[:2])

def extract_gz_file(gz_file, output_file):
    if not os.path.exists(output_file):
        with gzip.open(gz_file, 'rb') as f_in:
            with open(output_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

def scale_and_save_image(input_file, scaled_data, output_dir):
    try:
        img = nib.load(input_file)
        scaled_img = nib.Nifti1Image(scaled_data, affine=img.affine)
        scaled_file_name = os.path.join(output_dir, os.path.basename(input_file).replace('.nii', '_inst_edema_scaled.nii.gz'))
        nib.save(scaled_img, scaled_file_name)
    except Exception as e:
        print(f"Error processing file {input_file}: {e}")

root_dir = '/Users/yehudadicker/Library/Mobile Documents/com~apple~CloudDocs/Penn_Research'  
institutions = ['Penn', 'CWRU', 'NYU', 'TJU', 'OSU', 'RH']


for inst in institutions:
    baseline_dir = os.path.join(root_dir, inst, 'Baseline')
    seg_dir = os.path.join(root_dir, inst, 'Seg')
    output_dir = '/Users/yehudadicker/Library/Mobile Documents/com~apple~CloudDocs/Penn_Research/Scaled_Images/Min_Max_Scaled/Edema_Fit/By_Institution'
    output_dir = os.path.join(output_dir, inst)
    os.makedirs(output_dir, exist_ok=True)

    baseline_files = glob.glob(f"{baseline_dir}/*.nii.gz")
    all_edema_voxels = np.array([])

    # Collect all edema voxels
    for file in baseline_files:
        extracted_file = file[:-3]
        common_prefix = extract_common_prefix(file)
        segmentation_filename = f"{common_prefix}_LPS_rSRI_segmF.nii"
        segmentation_file = os.path.join(seg_dir, segmentation_filename)

        if not os.path.exists(segmentation_file):
            zipped_segmentation_file = segmentation_file + '.gz'
            if os.path.exists(zipped_segmentation_file):
                extract_gz_file(zipped_segmentation_file, segmentation_file)
            else:
                print(f"No matching segmentation file for {file}, skipping.")
                continue

        edema_voxels = get_edema_voxels(extracted_file, segmentation_file)
        if edema_voxels.size > 0:
            all_edema_voxels = np.concatenate((all_edema_voxels, edema_voxels)) if all_edema_voxels.size else edema_voxels

    # Fit the scaler to the combined edema voxels
    if all_edema_voxels.size > 0:
        scaler = MinMaxScaler()
        scaler.fit(all_edema_voxels)

        # Scale each image
        for file in baseline_files:
            extracted_file = file[:-3]
            common_prefix = extract_common_prefix(file)
            segmentation_filename = f"{common_prefix}_LPS_rSRI_segmF.nii"
            segmentation_file = os.path.join(seg_dir, segmentation_filename)

            if os.path.exists(segmentation_file):
                img = nib.load(extracted_file)
                data = img.get_fdata()
                scaled_data = scaler.transform(data.reshape(-1, 1)).reshape(data.shape)
                scale_and_save_image(extracted_file, scaled_data, output_dir)
            else:
                print(f"Segmentation file missing for {file}, skipping.")
    else:
        print(f"No edema data found for institution {inst}, skipping.")
